In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90,
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_data_scaled =  StandardScaler().fit_transform(df_market_data[["price_change_percentage_24h","price_change_percentage_7d",
                                                                     "price_change_percentage_14d", "price_change_percentage_30d",
                                                                    "price_change_percentage_60d","price_change_percentage_200d", 
                                                                     "price_change_percentage_1y"]])

In [ ]:
# Create a DataFrame with the scaled data
market_data_scaled_df = pd.DataFrame(market_data_scaled, columns=["price_change_percentage_24h","price_change_percentage_7d",
                                                                     "price_change_percentage_14d", "price_change_percentage_30d",
                                                                    "price_change_percentage_60d","price_change_percentage_200d", 
                                                                     "price_change_percentage_1y"])

# Copy the crypto names from the original data
market_data_scaled_df["coinid"] = df_market_data.index
# Set the coinid column as index
market_data_scaled_df.set_index("coinid", inplace=True)

# Display sample data
market_data_scaled_df.head()

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,11))

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state=2)
    k_model.fit(market_data_scaled_df)
    inertia.append(k_model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_original_elbow = pd.DataFrame(elbow_data)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_original_elbow.hvplot.line(x='k', y='inertia', title='Original Elbow Curve')

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [ ]:
# Fit the K-Means model using the scaled data
model.fit(market_data_scaled_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
k_predict = model.predict(market_data_scaled_df)

# Print the resulting array of cluster values.
k_predict

In [ ]:
# Create a copy of the DataFrame
market_predictions_df = market_data_scaled_df.copy()


In [ ]:
# Add a new column to the DataFrame with the predicted clusters
market_predictions_df['cluster']=k_predict


# Display sample data
market_predictions_df.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
original_plot=market_predictions_df.hvplot.scatter(x='price_change_percentage_24h', y='price_change_percentage_7d',
                                                   by='cluster', hover_cols ='coinid', marker = '*', size = 50)
original_plot

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_predictions_pca = pca.fit_transform(market_data_scaled_df)
# View the first five rows of the DataFrame. 
market_predictions_pca[0:5]

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 0.89503166

In [ ]:
# Create a new DataFrame with the PCA data.
# Creating a DataFrame with the PCA data
market_predictions_pca_df = pd.DataFrame(market_predictions_pca, columns=['PC1', 'PC2', 'PC3'])

# Copy the crypto names from the original data
market_predictions_pca_df['coinid']=market_predictions_df.index

# Set the coinid column as index
market_predictions_pca_df.set_index('coinid', inplace=True)

# Display sample data
market_predictions_pca_df.head()

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1,11))

In [ ]:
# Create an empty list to store the inertia values
inertia=[]

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
for i in k_values:
    K_model = KMeans(n_clusters=i, random_state =1)
    K_model.fit(market_predictions_pca_df)
    inertia.append(K_model.inertia_)
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `market_predictions_pca_df`
# 3. Append the model.inertia_ to the inertia list


In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k_values, "inertia": inertia}
# Create a DataFrame with the data to plot the Elbow curve
pca_elbow_df = pd.DataFrame(elbow_data)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
pca_elbow_df.hvplot.line(y='inertia', x='k', title='PCA Elbow Curve')

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** It does not

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
PCAModel = KMeans(n_clusters=4, random_state=1)

In [ ]:
# Fit the K-Means model using the PCA data
PCAModel.fit(market_predictions_pca_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
PCAclusters = PCAModel.predict(market_predictions_pca_df)
# Print the resulting array of cluster values.
PCAclusters

In [ ]:
# Create a copy of the DataFrame with the PCA data
market_predicted_pca_df = market_predictions_pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
market_predicted_pca_df['clusters'] = PCAclusters

# Display sample data
market_predicted_pca_df.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_plot = market_predicted_pca_df.hvplot.scatter(x='PC1', y='PC2', marker='square', by='clusters', hover_cols = 'coinid', alpha=0.5, size=80)
pca_plot

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
composite_elbow_df=df_original_elbow.copy()
composite_elbow_df['Original Inertia']=composite_elbow_df['inertia']
composite_elbow_df['PCA Inertia']=pca_elbow_df['inertia']
composite_elbow_df.drop(columns=['inertia'], inplace=True)
composite_elbow_df[['PCA Inertia', 'Original Inertia','k']].hvplot.line(x='k', ylabel='Inertia', title='Comparative Inertia')

In [ ]:
# Composite plot to contrast the clusters
pca_plot * original_plot

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** As evidenced by the elbow curves, the PCA data has less inertia than the orginal dataset indicating less point dispersion within the clusters.  This decrease in disperison suggests the clusters are more homgeneous than they are actually.